EJERCICIO 1

In [7]:
import pandas as pd
import numpy as np
import random
import os
from datetime import datetime

class Paciente:
    def __init__(self, id, saturacion_oxigeno, vacunas):
        self.id = id
        self.saturacion_oxigeno = saturacion_oxigeno
        self.vacunas = vacunas
        self.prueba_covid = None
        self.destino = None

    def realizar_prueba_covid(self):
        # Simular una prueba de COVID (positiva o negativa)
        self.prueba_covid = random.choice([True, False])
        return self.prueba_covid

    def determinar_destino(self, camillas_disponibles):
        # Criterio A: Saturación de oxígeno por debajo de 90
        if self.saturacion_oxigeno < 90:
            self.destino = "Sala COVID"
            return "Sala COVID"

        # Criterio B: Menos de 2 vacunas
        if self.vacunas < 2:
            self.destino = "Sala COVID"
            return "Sala COVID"

        # Criterio C: Prueba de COVID si los criterios anteriores no son concluyentes
        if self.prueba_covid is None:
            self.realizar_prueba_covid()

        if self.prueba_covid:
            self.destino = "Sala COVID"
            return "Sala COVID"

        # Criterio D y E: Si ninguna de las tres preguntas anteriores es positiva
        # y hay camillas disponibles
        if camillas_disponibles > 0:
            self.destino = "Urgencias"
            return "Urgencias"
        else:
            self.destino = "En Espera"
            return "En Espera"

class SistemaTriage:
    def __init__(self, num_pacientes=50, camillas_disponibles=10):
        self.num_pacientes = num_pacientes
        self.camillas_disponibles = camillas_disponibles
        self.pacientes = []
        self.generar_pacientes()

    def generar_pacientes(self):
        # Generar datos aleatorios para los pacientes
        for i in range(1, self.num_pacientes + 1):
            # Saturación de oxígeno entre 80 y 100
            saturacion = random.randint(80, 100)
            # Número de vacunas entre 0 y 4
            vacunas = random.randint(0, 4)
            self.pacientes.append(Paciente(i, saturacion, vacunas))

    def ejecutar_triage(self):
        resultados = []

        for paciente in self.pacientes:
            destino = paciente.determinar_destino(self.camillas_disponibles)

            # Reducir el número de camillas disponibles si el paciente va a urgencias
            if destino == "Urgencias":
                self.camillas_disponibles -= 1

            resultados.append({
                'ID': paciente.id,
                'Saturacion_Oxigeno': paciente.saturacion_oxigeno,
                'Vacunas': paciente.vacunas,
                'Prueba_COVID': paciente.prueba_covid,
                'Destino': destino
            })

        return pd.DataFrame(resultados)

    def guardar_datos(self, filename="datos_pacientes.csv"):
        # Crear un dataframe con los datos de entrada
        datos = []
        for p in self.pacientes:
            datos.append({
                'ID': p.id,
                'Saturacion_Oxigeno': p.saturacion_oxigeno,
                'Vacunas': p.vacunas
            })

        df = pd.DataFrame(datos)
        df.to_csv(filename, index=False)
        return filename

    def guardar_resultados(self, resultados, filename="resultados_triage.csv"):
        # Guardar resultados en un archivo protegido
        resultados.to_csv(filename, index=False)
        return filename

# Ejecutar el sistema
if __name__ == "__main__":
    # Crear el sistema de triage
    sistema = SistemaTriage()

    # Guardar los datos de entrada generados aleatoriamente
    archivo_entrada = sistema.guardar_datos()
    print(f"Datos de entrada guardados en: {archivo_entrada}")

    # Ejecutar el triage y obtener resultados
    resultados = sistema.ejecutar_triage()

    # Guardar los resultados
    archivo_resultados = sistema.guardar_resultados(resultados)
    print(f"Resultados guardados en: {archivo_resultados}")

    # Mostrar estadísticas
    print("\nEstadísticas:")
    print(f"Total de pacientes: {len(resultados)}")
    print(f"Pacientes en Sala COVID: {sum(resultados['Destino'] == 'Sala COVID')}")
    print(f"Pacientes en Urgencias: {sum(resultados['Destino'] == 'Urgencias')}")
    print(f"Pacientes en Espera: {sum(resultados['Destino'] == 'En Espera')}")

Datos de entrada guardados en: datos_pacientes.csv
Resultados guardados en: resultados_triage.csv

Estadísticas:
Total de pacientes: 50
Pacientes en Sala COVID: 39
Pacientes en Urgencias: 10
Pacientes en Espera: 1


EJERCICIO 2

In [8]:
import pandas as pd
import numpy as np
import random
import os
from datetime import datetime

class Persona:
    def __init__(self, id):
        self.id = id
        # Generar características aleatorias para la persona
        self.tiene_tarjetas = random.choice([True, False])
        self.reportado_riesgo = random.choice([True, False])
        self.tipo_contrato = random.choice(["Indefinido", "Definido", "Ninguno"])
        self.credito_otra_entidad = random.choice([True, False])
        self.credito_misma_entidad = random.choice([True, False])
        self.edad = random.randint(18, 70)
        self.puntuacion = 0
        self.aprobado = False

    def calcular_puntuacion(self):
        # Reiniciar puntuación
        self.puntuacion = 0

        # a. 1 punto: Si posee tarjetas con otras franquicias
        if self.tiene_tarjetas:
            self.puntuacion += 1

        # b. 3 puntos: Si NO está reportado ante entidades de riesgo
        if not self.reportado_riesgo:
            self.puntuacion += 3

        # c. 2 puntos: Si se encuentra laborando a término definido
        if self.tipo_contrato == "Definido":
            self.puntuacion += 2

        # d. 3 puntos: Si se encuentra laborando a término indefinido
        if self.tipo_contrato == "Indefinido":
            self.puntuacion += 3

        # e. 2 puntos: Si ya había obtenido créditos con anterioridad en otra entidad bancaria
        if self.credito_otra_entidad:
            self.puntuacion += 2

        # f. 3 puntos: Si ya había obtenido crédito en la misma entidad bancaria
        if self.credito_misma_entidad:
            self.puntuacion += 3

        # Puntos por edad
        # g. 2 puntos: Si la edad de la persona oscila entre los 18 a los 22 años
        if 18 <= self.edad <= 22:
            self.puntuacion += 2
        # h. 4 puntos: Si la edad de la persona oscila entre los 23 y 30 años
        elif 23 <= self.edad <= 30:
            self.puntuacion += 4
        # i. 3 puntos: Si la edad de la persona oscila entre los 31 y 40 años
        elif 31 <= self.edad <= 40:
            self.puntuacion += 3
        # j. 1 punto: Si la edad de la persona es mayor a 41 años
        elif self.edad >= 41:
            self.puntuacion += 1

        # Verificar si el crédito es aprobado (puntuación >= 6)
        self.aprobado = self.puntuacion >= 6

        return self.puntuacion, self.aprobado

class SistemaCredito:
    def __init__(self, num_personas=100):
        self.num_personas = num_personas
        self.personas = []
        self.generar_personas()

    def generar_personas(self):
        # Generar datos aleatorios para las personas
        for i in range(1, self.num_personas + 1):
            self.personas.append(Persona(i))

    def evaluar_creditos(self):
        resultados = []

        for persona in self.personas:
            puntuacion, aprobado = persona.calcular_puntuacion()

            resultados.append({
                'ID': persona.id,
                'Edad': persona.edad,
                'Tiene_Tarjetas': persona.tiene_tarjetas,
                'Reportado_Riesgo': persona.reportado_riesgo,
                'Tipo_Contrato': persona.tipo_contrato,
                'Credito_Otra_Entidad': persona.credito_otra_entidad,
                'Credito_Misma_Entidad': persona.credito_misma_entidad,
                'Puntuacion': puntuacion,
                'Credito_Aprobado': aprobado
            })

        return pd.DataFrame(resultados)

    def guardar_datos(self, filename="datos_personas.csv"):
        # Crear un dataframe con los datos de entrada
        datos = []
        for p in self.personas:
            datos.append({
                'ID': p.id,
                'Edad': p.edad,
                'Tiene_Tarjetas': p.tiene_tarjetas,
                'Reportado_Riesgo': p.reportado_riesgo,
                'Tipo_Contrato': p.tipo_contrato,
                'Credito_Otra_Entidad': p.credito_otra_entidad,
                'Credito_Misma_Entidad': p.credito_misma_entidad
            })

        df = pd.DataFrame(datos)
        df.to_csv(filename, index=False)
        return filename

    def guardar_resultados(self, resultados, filename="resultados_credito.csv"):
        # Guardar resultados en un archivo protegido
        resultados.to_csv(filename, index=False)
        return filename

# Ejecutar el sistema
if __name__ == "__main__":
    # Crear el sistema de evaluación de créditos
    sistema = SistemaCredito()

    # Guardar los datos de entrada generados aleatoriamente
    archivo_entrada = sistema.guardar_datos()
    print(f"Datos de entrada guardados en: {archivo_entrada}")

    # Evaluar créditos y obtener resultados
    resultados = sistema.evaluar_creditos()

    # Guardar los resultados
    archivo_resultados = sistema.guardar_resultados(resultados)
    print(f"Resultados guardados en: {archivo_resultados}")

    # Mostrar estadísticas
    print("\nEstadísticas:")
    print(f"Total de personas evaluadas: {len(resultados)}")
    print(f"Personas con crédito aprobado: {sum(resultados['Credito_Aprobado'])}")
    print(f"Personas con crédito rechazado: {sum(~resultados['Credito_Aprobado'])}")
    print(f"Porcentaje de aprobación: {sum(resultados['Credito_Aprobado'])/len(resultados)*100:.2f}%")

Datos de entrada guardados en: datos_personas.csv
Resultados guardados en: resultados_credito.csv

Estadísticas:
Total de personas evaluadas: 100
Personas con crédito aprobado: 73
Personas con crédito rechazado: 27
Porcentaje de aprobación: 73.00%
